# Make Predictions for House Prices with SAP AI Core

Author: https://github.com/dhrubpaul

You need Docker to complete this tutorial. If you are running this Jupyter notebook on web, we recommend to use your local system along with it.

The steps are analogous with the tutorial: https://developers.sap.com/tutorials/ai-core-deploy.html
Please open the tutorial and this notebook side-by-side for better understanding.

## Pre-requisite 
Create connection with SAP AI Core. Edit the below cell

In [ ]:
# Load Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client(
    base_url = "<YOUR_AI_API_URL>" + "/v2", # The present SAP AI Core API version is 2
    auth_url=  "<YOUR_url>" + "/oauth/token", # Suffix to add
    client_id = "<YOUR_clientid>",
    client_secret = "<YOUR_clientsecret>"
)
# no output is expected

## Step 1: Write code for serving engine

*Refer step 1 of the tutorial: https://developers.sap.com/tutorials/ai-core-deploy.html#7884d459-518e-433c-b694-59ddca782ebd*

## Step 2: Bundle and publish code to cloud

*Refer step 2 of the tutorial: https://developers.sap.com/tutorials/ai-core-deploy.html#c897573a-ce36-4a2c-a32d-a7d2d505caff*

## Step 3: Create a serving executable

*Refer step 3 of the tutorial: https://developers.sap.com/tutorials/ai-core-deploy.html#b4a49a57-b260-4389-83ba-1675cc84a85d*

## Step 4: Select a model to deploy using a configuration

In [ ]:
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding

response = ai_core_client.configuration.create(
    name = "House Price Predictor Jan",
    scenario_id = "learning-datalines",
    executable_id = "server-pipeline",
    input_artifact_bindings = [
        InputArtifactBinding(key = "housepricemodel", artifact_id = "<YOUR_MODEL_ARTIFACT_ID>") # placeholder  name as key
    ],
    parameter_bindings = [
        ParameterBinding(key = "greetmessage", value = "Hi AI Core server") # placeholder name as key
    ],
    resource_group = "default"
)
print(response.__dict__)


## Step 5: Start a deployment

In [ ]:
response = ai_core_client.deployment.create(
    configuration_id="YOUR_CONFIGURATIO_ID",
    resource_group='default'
)

print(response.__dict__)


## Step 6: Make a prediction


In [ ]:
# Get Deployment details
response = ai_core_client.deployment.get(
    deployment_id="YOUR_DEPLOYMENT_ID",
    resource_group="default"
)

print(response.__dict__)


In [ ]:
# Make Prediction
import requests

# URL
deployment_url = "YOUR_DEPLOYMENT_URL"

# Preparing the input for inference
test_input = {
    "MedInc": 8.3252,
    "HouseAge": 41,
    "AveRooms": 6.984126984,
    "AveBedrms":1.023809524,
    "Population":322,
    "AveOccup": 2.555555556,
    "Latitude":37.88,
    "Longitude":-122.23
}

endpoint = f"{deployment_url}/v2/predict" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "default",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)

print('Inference result:', response.json())


## Step 7: Switch the deployed model

In [ ]:
# Create a new configuration
response = ai_core_client.configuration.create(
    name = "House Price Predictor Feb",
    scenario_id = "learning-datalines",
    executable_id = "server-pipeline",
    input_artifact_bindings = [
        InputArtifactBinding(key = "housepricemodel", artifact_id = "<YOUR_MODEL_ARTIFACT_ID>") # placeholder  name as key
    ],
    parameter_bindings = [
        ParameterBinding(key = "greetmessage", value = "Hi AI Core server") # placeholder name as key
    ],
    resource_group = "default"
)
print(response.__dict__)


In [ ]:
# Modify deployment to use new configuration
response = ai_core_client.deployment.modify(
    deployment_id = "YOUR_DEPLOYMENT_ID", # existing deployment
    configuration_id = "NEW_CONFIGURATION_ID", # new configuration ID
    resource_group = "default"
)

print(response.__dict__)


## Step 8: Stop a deployment

In [ ]:
from ai_api_client_sdk.models.target_status import TargetStatus
response = ai_core_client.deployment.modify(
    deployment_id="YOUR_DEPLOYMENT_ID",
    target_status=TargetStatus.STOPPED,
    resource_group="default"
)

print(response.__dict__)
